In [ ]:
!pip install pennylane
!pip install openfermion
!pip install openfermionpyscf

In [ ]:
import openfermion
import openfermion.ops.representations as reps
import pennylane as qml

from openfermionpyscf import run_pyscf
from openfermion.transforms import get_fermion_operator , jordan_wigner
from pennylane import numpy as np


H_position = 1.32
conversion_units = 0.52917721067121
H_position_after_conversion = conversion_units * H_position

geometry = [("Be", [0, 0, 0]), ('H', [0, 0, -H_position_after_conversion]),('H', [0, 0, H_position_after_conversion])]
basis = 'sto3g'
multiplicity = 1
charge = 0
beh2molecule = openfermion.chem.MolecularData(geometry , basis , multiplicity , charge)

beh2molecule_calculated = run_pyscf(beh2molecule , run_mp2=True , run_cisd=True, run_ccsd=True, run_fci=True)

constant = beh2molecule_calculated.nuclear_repulsion
one_body_integrals = beh2molecule_calculated.one_body_integrals
two_body_integrals = beh2molecule_calculated.two_body_integrals

one_body_coefficients, two_body_coefficients = openfermion.chem.molecular_data.spinorb_from_spatial(one_body_integrals, two_body_integrals)

molecular_hamiltonian = reps.InteractionOperator(constant, one_body_coefficients, 1/2*two_body_coefficients)
# 1/2*two_body_coefficients
# np.zeros(two_body_coefficients.shape)

beh2_qubit_hamiltonian = jordan_wigner(get_fermion_operator(molecular_hamiltonian))

beh2_qubit_hamiltonian_pennylane = qml.import_operator(beh2_qubit_hamiltonian, format='openfermion')

print(beh2_qubit_hamiltonian_pennylane)